## Numerical Analysis - Fall semester 2024

# Serie 13 - Adaptive ODE solver and boundary value problem

Package imports.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

<hr style="clear:both">

### ODE solver with adaptive step size

Let us consider the ordinary differential equation:

$$
\begin{cases}
\displaystyle \frac{\mathrm{d}u(t)}{\mathrm{d}t} = f(t,u(t)) \quad t \in (0,T], \\
u(0)=u_0
\end{cases}
$$

The goal is to compute an approximation of the solution with a certain given tolerance $tol$ using an adaptive step size, meaning $\Delta t_n$ can change in every step. One such approach combines the forward Euler and Heun methods. It uses the forward Euler method to compute the approximation

$$
u^{n+1} = u^n + \Delta t_n f(t_n, u^n),
$$

and checks if the approximation error

$$
\frac{|u(t_{n+1}) - u^{n+1}|}{\Delta t_n}
$$

is compared with $tol / T$, and the step size decreased or increased accordingly (see lecture notes for details). However, since $u(t_{n+1})$ is not known, it is replaced with

$$
\hat{u}^{n+1} = u^{n} + \frac{\Delta t_n}{2} f(t_n, u^n) + \frac{\Delta t_n}{2} f(t_n + \Delta t_n, u^n + \Delta t_n f(t_n, u^n));
$$
the more accurate approximation from Heun's method.

<div class="alert alert-success">
    
**Exercise 1:** Complete the implementation of the ODE solver with adaptive step size (Algorithm 6.1 in the lecture notes) in the function `adaptive` below.
</div>

<div class="alert alert-warning">
    
**Warning:** The function `adaptive` returns the array `u_hat`, which contains the Heun's method approximations $[\hat{u}^{0}, \hat{u}^{1}, \hat{u}^{2}, \dots]$. An approximation $\hat{u}^{n+1}$ is only added to this array, once a step size $\Delta t_n$ has been found with which the Euler's method verifies the tolerance. Otherwise the approximation is discarded and the step size $\Delta t_n$ is decreased for an anew computation of the forward Euler and Heun's method approximations.
</div>

In [ ]:
def adaptive(f, u_0, T, dt_init, tol):
    # Set initial parameters and arrays
    n = 0
    dt = min(T, dt_init)
    t = [0]
    u = [u_0]
    u_hat = [u_0]
    
    while t[n] < T:
        # YOUR CODE HERE
        raise NotImplementedError()
    return np.array(t), np.array(u_hat)

<div class="alert alert-success">
    
**Exercise 2:** Use your implementation of the adaptive ODE solver to approximate the ODE from above with the data

$$
f(t,u)=t(1-u)+(1-t)e^{-t}, \quad u_0=1, \quad T=10, \quad \Delta t_{\text{init}} = 0.1, \quad tol=10^{-1},
$$

whose exact solution is given by $u(t)=e^{-t^2/2}-e^{-t}+1$. Plot the obtained solution with `plt.scatter(t, u)` and compare it with the exact solution. When do you observe smaller step sizes and when larger ones?
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-success">
    
**Exercise 3:** Repeat the previous exercise with

$$
f(t,u)=1+\frac{e^{t/\varepsilon}}{\varepsilon(1-e^{1/\varepsilon})}, \quad u_0=0, \quad T=1, \quad \Delta t_{\text{init}} = 0.1, \quad tol=10^{-3}
$$
whose exact solution is given by $u(t)=t-\frac{1-e^{t/\varepsilon}}{1-e^{1/\varepsilon}}$, for $\varepsilon=0.1,0.01$.
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<hr style="clear:both">

### Dirichlet vs mixed boundary conditions

We consider the bonudary value problem

$$
\begin{cases}
\displaystyle \frac{\partial^2 u}{\partial x^2}(x) = u(x), \qquad x \in (0, 1), \\
u(0) = \alpha,~u(1) = \beta.
\end{cases}
$$

After replacing the derivative with a second order finite difference approximation at the $n + 2$ evenly spaced nodes $0 = x_0 < x_1 < \cdots < x_{n + 1} = 1$, we get the discretized problem

$$
\begin{cases}
\displaystyle \frac{u_{j-1} - 2 u_j + u_{j + 1}}{h^2} = u_j, \qquad j = 1, 2, \dots, n \\
u_0 = \alpha,~u_{n + 1} = \beta
\end{cases}
$$

This can be simplified to

$$
\begin{cases}
\displaystyle \frac{u_{j-1} - (2 + h^2) u_j + u_{j + 1}}{h^2} = 0, \qquad j = 1, 2, \dots, n \\
u_0 = \alpha,~u_{n + 1} = \beta
\end{cases}
$$

and compactly written as a system

$$
A \mathbf{u} = \mathbf{f}
$$

where $\mathbf{u} = [u^{1}, u^{2}, \dots, u^{n}]^{\top}$ is the unknown solution, $\mathbf{f} = [\alpha / h^2, 0, \dots, 0, \beta / h^2]^{\top}$, and

$$
A = \frac{1}{h^2} 
\begin{pmatrix} 
    2 + h^2 & -1 & 0 & \cdots & 0 \\
    -1 & 2 + h^2 & \ddots &  & \vdots \\
    0 & \ddots & \ddots & -1 & 0 \\
    \vdots &  & -1 & 2 + h^2 & -1 \\
    0 & \cdots & 0 & -1 & 2 + h^2 \\
\end{pmatrix} \in \mathbb{R}^{n \times n}.
$$

The function `solve_dirichlet` computes the finite difference approximation `u`, i.e. an array whose entries contain the approximations $u^{j} \approx u(x_i), j= 0, 1, \dots, n+1$ of the solution for some value of $\alpha$ and $\beta$.

In [ ]:
def solve_dirichlet(n, alpha, beta):
    h = 1 / (n + 1)
    
    # Set up system matrix and right-hand side
    A = 1 / h ** 2 * (- np.diag(np.ones(n - 1), -1) + np.diag((2 + h ** 2) * np.ones(n)) - np.diag(np.ones(n - 1), 1))
    f = np.zeros(n)
    f[0] = alpha / h ** 2
    f[-1] = beta / h ** 2

    # Compose solution
    u = np.zeros(n + 2)
    u[1:-1] = np.linalg.solve(A, f)
    u[0] = alpha
    u[-1] = beta
    return u

<div class="alert alert-success">
    
**Exercise 4:** For $n = 2^k, k=2, 3, \dots, 10$, use the function `solve_dirichlet` to solve the Dirichlet boundary value problem. Take $\alpha = 0$ and $\beta = 1$, and compute the error $\max_{j = 0, 1, \dots, n+1} |u(x_j) - u^j|$ of the approximate solution $u^j$ to the exact solution $u(x) = \sinh(x) / \sinh(1)$ (use the NumPy function `np.sinh`). Plot the error in terms of $h$. Is this consistent with Theorem 7.2 in the lecture notes? 
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Now consider the new problem

$$
\begin{cases}
\displaystyle \frac{\partial^2 u}{\partial x^2}(x) = u(x), \qquad x \in (0, 1), \\
u(0) = \alpha,~u(1) + u'(1) = \beta,
\end{cases}
$$

where merely one of the boundary conditions changed to a mixed boundary condition in terms of the function value and its derivative.


<div class="alert alert-success">
    
**Exercise 5 (Theoretical):** Derive the linear system $A \mathbf{u} = \mathbf{f}$ whose solution is the finite difference approximation of the system.

*Hint:* Replace the term $u'(1)$ by its backward finite differences approximation. 
</div>


<div class="alert alert-success">
    
**Exercise 6:** Implement the function `solve_mixed`, which assembles the matrix $A$ and the vector $\mathbf{f}$, and uses them to compute the approximate finite differences solution.

*Hint:* You can copy the code from the function `solve_dirichlet`, and will just have to adjust a few lines.
</div>

In [ ]:
def solve_mixed(n, alpha, beta):
    # YOUR CODE HERE
    raise NotImplementedError()
    return u

<div class="alert alert-success">
    
**Exercise 7:** For $n = 2^k, k=2, 3, \dots, 10$, use the function `solve_mixed` you have implemented in the previous exercise to solve the boundary value problem. Take $\alpha = 0$ and $\beta = e / \sinh(1)$, and compute the error $\max_{j = 0, 1, \dots, n+1} |u(x_j) - u^j|$ of the approximate solution $u^j$ to the exact solution $u(x) = \sinh(x) / \sinh(1)$. Plot the error in terms of $h$. What order of convergence do you observe?  Why?
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<hr style="clear:both">

### Modelling Earth's surface temperature

We want to simulate the temperature of the Earth's surface until a depth of $5$ meters during four days ($96$ hours). The temperature at a time $t$ (in hours) and a depth $x$ (in meters) is denoted with $T(t, x)$ (in degree Celsius). We assume that at a depth of $5$ m, the temperature is constant and equal to $T(t, x = 5~\mathrm{m}) = 0~^{\circ} \mathrm{C}$. The top of the surface is periodically heated by the sun during the day and cools off during the night. This is modelled with $T(t, x = 0~\mathrm{m}) = \hat{T} \cos(\omega_0 t)$ where $\omega_0 = \pi / 12~\mathrm{h}^{-1}$  and $\hat{T} = 10~^{\circ} \mathrm{C}$. We assume that at the start of the simulation, the temperature at any depth is $0~^{\circ} \mathrm{C}$, i.e. $T(t = 0~\mathrm{h}, x) = 0~^{\circ} \mathrm{C}$.

The evolution of the temperature $T(t, x)$ can is modelled by the heat equation

$$
\frac{\partial T(t, x)}{\partial t} = \alpha \frac{\partial^2 T(t, x)}{\partial x^2}.
$$

where $\alpha = 0.03~\mathrm{m}^2 / \mathrm{s}$ is the diffusion coefficient of the Earth's surface.

To approximate the solution to the heat equation, we discretize the time interval $[0, 96]$ into $n_t + 2$ time steps $t_j, j = 0, 1, 2, \dots, n_t + 1$, separated by $\Delta t$, and the depth interval $[0, 5]$ into $n_x + 2$ depth steps $x_i, i = 0, 1, 2, \dots, n_x + 1$, separated by $\Delta x$, and use the finite differences approximation to compute the temperature values $T(t_j, x_i)$ on the grid $\{(t_j = j \Delta t, x_i = i \Delta x), j = 0, 1, 2, \dots, n_t + 1, i = 0, 1, 2, \dots, n_x + 1\}$.

<div class="alert alert-success">
    
**Exercise 8 (Theoretical):** Write down the discretized heat equation on the grid. That is, replace the derivative $\frac{\partial T(t_j, x_i)}{\partial t}$ by its forward finite differences approximation and $\frac{\partial^2 T(t_j, x_i)}{\partial x^2}$ by its standard finite differences approximations in terms of their adjacent grid points.
</div>


<div class="alert alert-success">
    
**Exercise 9 (Theoretical):** Rearrange the discretized heat equation from the previous exercise for $T(t_{j+1}, x_i)$, such that it reads

$$
T(t_{j+1}, x_i) = \gamma T(t_j, x_{i - 1}) + (1 - 2 \gamma) T( t_j, x_i) + \gamma T(t_j, x_{i + 1})
$$

for some constant $\gamma$, which you should express in terms of $\alpha$, $\Delta t$, and $\Delta x$. 
</div>


Thanks to the formula from Exercise 5, and thanks to the boundary conditions, we can now determine the temperature at all times and  depths on the grid. To do this, we start from $t_0$, and fill in the temperature values for the depths $x_i, i=1, 2, \dots, n_x$ at $t_1$ using the formula from Exercise 5, and then proceed in the same manner for $t_2, t_3, \dots$ Running the below cell will provide you with a schematic illustration of what this process looks like.

In [ ]:
fig, ax = plt.subplots(); ax.axis("off"); t = np.linspace(-0.25, 0.75, 9); x = np.linspace(0, 1, 7); tv, xv = np.meshgrid(t, x); ax.scatter(tv, xv, s=16, color="black", label="boundary values"); t = np.linspace(0, 0.75, 7); x = np.linspace(0.168, 0.668, 4); tv, xv = np.meshgrid(t, x); ax.scatter(tv, xv, s=25, color="red", label="unknown"); plt.legend(); ax.add_patch(mpl.patches.Circle((0, 0.67), 0.03, fill=False, edgecolor="black", linewidth=2)); ax.add_patch(mpl.patches.Circle((0, 0.5), 0.03, fill=False, edgecolor="black", linewidth=2)); ax.add_patch(mpl.patches.Circle((0, 0.33), 0.03, fill=False, edgecolor="black", linewidth=2)); ax.add_patch(mpl.patches.Circle((0.125, 0.5), 0.03, fill=False, edgecolor="black", linewidth=2)); ax.add_patch(mpl.patches.Arrow(0, 0.67, 0.12, -0.12, color="black", width=0.05)); ax.add_patch(mpl.patches.Arrow(0, 0.33, 0.12, 0.12, color="black", width=0.05)); ax.add_patch(mpl.patches.Arrow(0, 0.5, 0.09, 0, color="black", width=0.05)); ax.text(-0.04, 0.57, "$T(t_j, x_i)$", fontsize=12, horizontalalignment="center", verticalalignment="center"); ax.text(-0.06, 0.4, "$T(t_j, x_{i+1})$", fontsize=12, horizontalalignment="center", verticalalignment="center"); ax.text(-0.06, 0.74, "$T(t_j, x_{i-1})$", fontsize=12, horizontalalignment="center", verticalalignment="center"); ax.text(0.22, 0.56, "$T(t_{j+1}, x_i)$", fontsize=12, horizontalalignment="center", verticalalignment="center"); ax.add_patch(mpl.patches.Arrow(-0.25, 1, 0, -1.1, color="black", width=0.1)); ax.add_patch(mpl.patches.Arrow(-0.25, 1, 1.1, 0, color="black", width=0.1)); ax.text(0.8, 1.1, "$t$", fontsize=20, horizontalalignment="center", verticalalignment="center"); ax.text(-0.3, -0.1, "$x$", fontsize=20, horizontalalignment="center", verticalalignment="center"); 

Below, we provide you with the function `plot_temperature_profile` which takes as an input the $(n_t + 2) \times (n_x + 2)$ NumPy array $T$, whose entries are the temperatures $T[j, i] = T(t_j, x_i)$ on the grid, and visualizes this temperature evolution in a plot.

In [ ]:
def plot_temperature_profile(T, t_max=96, x_max=5):
    fig, ax = plt.subplots()

    n_t = T.shape[0] - 2
    n_x = T.shape[1] - 2
    
    im = plt.imshow(T.T, cmap="plasma", interpolation="gaussian")
    plt.colorbar(im, label=r"temperature $T$ ($^{\circ}$C)", orientation="horizontal")
    plt.xlabel(r"time $t$ (h)")
    plt.ylabel(r"depth $x$ (m)")
    plt.xticks(np.linspace(0, n_t + 1, 5).astype(int), np.linspace(0, t_max, 5).astype(int))
    plt.yticks(np.linspace(0, n_x + 1, 5).astype(int), np.linspace(0, x_max, 5).astype(int))

<div class="alert alert-success">
    
**Exercise 10:** Approximate the temperature $T(t_j, x_i)$ at the discrete times $t_j, j = 0, 1, 2, \dots, n_t + 1$ and depths $x_i, i = 0, 1, 2, \dots, n_x + 1$ with the procedure described above. Use $\Delta t = 1~\mathrm{h}$ and $\Delta x = 0.25~\mathrm{m}$. Visualize the approximation using the function `plot_temperature_profile`. Explain how the temperature changes throughout the day.

*Hint:* By working with the $(n_t + 2) \times (n_x + 2)$ NumPy array $T$, whose entries are $T[j, i] = T(t_j, x_i)$, the boundary condition at $t = 0~\mathrm{h}$ can easily be imposed through $T[0, :] = 0$, and similarly the boundary conditions at $x = 0~\mathrm{m}$ and $x = 5~\mathrm{m}$. The unknown entries can be filled in by looping over the array entries and using the formula from Exercise 5.
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<hr style="clear:both">

## The end

Congratulations! You have finished the last exercise notebook of the course.